Testing topic models with external documents

(i.e. comments from later games)

LDA

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import json
import pickle
from datetime import datetime
import traceback

import gensim
import nltk

import sys
sys.path.append('../')

In [2]:
%load_ext autoreload

In [61]:
# the text to be evaluated

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 1091500
# game_name = 'cyberpunk2077'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

game_steamid = 1716740
game_name = 'starfield'

datetime_until = datetime(2024, 1, 1, 0, 0, 0)      # only analyse reviews from this date until now (GMT+8)

# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'../../dataset/data_scraping/steam_comments_scraping/{game_name}').resolve()
if game_folder.exists():
    try:
        latest_file_path = sorted(game_folder.glob('steam_reviews_*.pkl'))[-1]
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

Loaded: /root/FYP/NLP/dev-workspace/dataset/data_scraping/steam_comments_scraping/starfield/steam_reviews_1716740_unique.pkl


In [62]:
# create a dataframe like in training/evaluation
reviews_df = pd.DataFrame(reviews_reqs)

reviews_df = reviews_df[['recommendationid', 'review', 'timestamp_created', 'voted_up', 'steam_purchase', 'received_for_free']]

# convert timestamp to datetime. The datetime converted is in utc+0
reviews_df['timestamp_created'] = pd.to_datetime(reviews_df['timestamp_created'], unit='s')

# convert the voted_up to 1 and -1
reviews_df['voted_up'] = reviews_df['voted_up'].apply(lambda x: 1 if x else -1)

reviews_df

,recommendationid,review,timestamp_created,voted_up,steam_purchase,received_for_free
0,157967184,I have loved every Bethesda game.\nI can't say...,2024-02-10 03:01:14,-1,True,False
1,157967139,Game just feels empty.. I don't enjoy replayin...,2024-02-10 03:00:23,-1,True,False
2,157966777,Was initially disappointed with this game but ...,2024-02-10 02:52:15,1,True,False
3,157965911,its kewl. its not some groundbreaking space ga...,2024-02-10 02:33:16,1,True,False
4,157964949,"7/10, they took “skyrim in space” a little too...",2024-02-10 02:15:34,1,True,False
...,...,...,...,...,...,...
96729,145736027,"65 hours played and I'm still loving it, and h...",2023-09-06 00:00:32,1,True,False
96730,145736019,This is my Skyrim.,2023-09-06 00:00:31,1,True,False
96731,145736018,It is fallout in space. Not some space SIM shit.,2023-09-06 00:00:31,1,False,False
96732,145736016,Starfield is a Bethesda RPG through and throug...,2023-09-06 00:00:30,1,False,False


In [63]:
%autoreload 2
sys.path.append('../../sa')
import str_cleaning_functions

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_non_letters(x))
    df[review] = df[review].apply(lambda x: x.lower())
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_stopword(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

def lemmatization_dataset(data):
    return {'review_text2': lemmatization(data['review_text'])}

In [64]:
# apply data preprocessing
cleaning(reviews_df, 'review')

from datasets import Dataset

temp_dataset = Dataset.from_dict({'review_text': reviews_df['review']})
temp_dataset = temp_dataset.map(lemmatization_dataset, num_proc=4)
reviews_df['review_lemmatized'] = temp_dataset['review_text2']

# filter empty list of strings in X_lemmatized, as they are not useful for topic modelling
# X_lemmatized = list(filter(lambda x: len(x) > 0, X_lemmatized))
reviews_df = reviews_df[reviews_df['review_lemmatized'].apply(len) > 0]
X_lemmatized = reviews_df['review_lemmatized'].values

print(len(X_lemmatized))
print(X_lemmatized[0])

Map (num_proc=4): 100%|██████████| 96734/96734 [00:35<00:00, 2702.86 examples/s]
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


95241
['love', 'every', 'bethesda', 'say', 'particularly', 'even', 'like', 'one']


In [65]:
# load the LDA model
%autoreload 2
from dataset_loader import GENRES

genre = GENRES.ACTION
training_datetime = datetime(2024, 2, 27, 9, 18, 50)
N_topics = 10

lda_model_folder = Path(f'../lda_dev/category_{str(genre)}_unique_review_text')
lda_model_folder = lda_model_folder.joinpath(
    Path(f'lda_multicore_genre_{str(genre)}_grid_search_{training_datetime.strftime("%Y%m%d_%H%M%S")}')
)
lda_model_folder = lda_model_folder.joinpath(
    Path(f'lda_multicore_lda_num_topics_{N_topics}')
)

# load the id2word and the model
id2word = gensim.corpora.Dictionary.load(str(lda_model_folder.joinpath('lda_multicore.id2word')))
lda_model = gensim.models.LdaMulticore.load(str(lda_model_folder.joinpath('lda_multicore')))

In [66]:
# create corpus object from the lemmatized reviews and id2word
corpus = [id2word.doc2bow(text) for text in X_lemmatized]

---

Evaluation copied from lda_eval_vis.ipynb and lda_eval_vis.ipynb

Evaluation

In [67]:
eval_results_external_folder_path = Path(f'../eval_results_external/{game_name}')

eval_results_external_folder_path = eval_results_external_folder_path.joinpath(
    *lda_model_folder.parts[2:]
)

print(eval_results_external_folder_path)

if not eval_results_external_folder_path.exists():
    eval_results_external_folder_path.mkdir(parents=True)

../eval_results_external/starfield/category_action_unique_review_text/lda_multicore_genre_action_grid_search_20240227_091850/lda_multicore_lda_num_topics_10


In [68]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.165960  0.318373       1        1  26.522600
8      0.050836  0.086453       2        1  22.755822
4     -0.106239 -0.006857       3        1  13.570137
1     -0.318944  0.205455       4        1  12.349739
3     -0.198515 -0.289504       5        1   6.806391
0     -0.074155  0.411493       6        1   4.935576
6      0.053157 -0.334076       7        1   4.342700
9     -0.377561 -0.123719       8        1   3.998610
7      0.314572 -0.306300       9        1   2.929578
5      0.490890  0.038681      10        1   1.788847, topic_info=           Term          Freq         Total Category  logprob  loglift
21         play  96665.000000  96665.000000  Default  10.0000  10.0000
9         great  26505.000000  26505.000000  Default   9.0000   9.0000
54          fun  42572.000000  42572.000000  Default   8.0000   8.0000
176        good  52141.000000  52141.000000  Default   7.0000   7.0000
8          like  74773.000000  74773.000000  Default   6.0000   6.0000
...         ...           ...           ...      ...      ...      ...
3348  storyline   1487.980984   4404.321200  Topic10  -3.9708   2.9384
178   recommend   3090.857626  16606.738988  Topic10  -3.2398   2.3422
200     graphic   3060.973144  19003.598713  Topic10  -3.2495   2.1977
10     gameplay   3072.012827  21127.110434  Topic10  -3.2459   2.0954
224        nice   1840.848143   8701.673147  Topic10  -3.7580   2.4703

[224 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
324       1  0.951494  ability
324       3  0.040117  ability
324       8  0.008409  ability
6397     10  0.998485       ac
376       1  0.253005   action
...     ...       ...      ...
398       5  0.079335      yes
398       6  0.003546      yes
398       7  0.053481      yes
398       8  0.002364      yes
5875      5  0.998892    zelda

[477 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 5, 2, 4, 1, 7, 10, 8, 6])

In [69]:
save_html = True
if save_html:
    pyLDAvis.save_html(vis, str(eval_results_external_folder_path.joinpath(f'pyldavis.html')))

---

Qualitative evaluation

Top 10 keywords for each topics

depends on the lda model, or the pyldavis

In [70]:
list(vis.sorted_terms(topic=1, _lambda=0.6)['Term'].values[:10])

['bioshock',
 'weapon',
 'character',
 'mission',
 'world',
 'different',
 'infinite',
 'fight',
 'level',
 'city']

In [71]:
topic_keywords = {}
topic_keywords_pyldavis = {}
top_N_words = 10

for i, topic in lda_model.show_topics(num_topics=lda_model.num_topics, num_words=top_N_words, formatted=False):
    topic_keywords[i] = [word for word, _ in topic]
    topic_keywords_pyldavis[i] = list(vis.sorted_terms(topic=i+1, _lambda=0.6)['Term'].values[:top_N_words])
    
    print(f'Topic {i}:')
    print(', '.join([word for word, _ in topic]))
    print(', '.join([word for word in topic_keywords_pyldavis[i]]))

    print()

Topic 0:
kill, shoot, gun, like, die, guy, shot, car, blow, na
bioshock, weapon, character, mission, world, different, infinite, fight, level, city

Topic 1:
pc, work, bad, issue, crash, problem, control, fix, bug, good
like, say, know, review, think, thing, sonic, come, play, end

Topic 2:
bioshock, weapon, character, world, level, mission, different, infinite, new, fight
gameplay, puzzle, level, plot, design, mechanic, experience, music, art, feel

Topic 3:
good, fun, like, really, play, pretty, cool, hard, nice, recommend
pc, work, issue, crash, fix, bug, bad, problem, version, control

Topic 4:
gameplay, good, level, play, puzzle, feel, character, graphic, really, experience
fun, good, really, like, pretty, play, cool, nice, funny, hard

Topic 5:
great, awesome, amazing, recommend, gameplay, graphic, highly, nice, storyline, fantastic
kill, shoot, gun, die, shot, car, na, explosion, guy, death

Topic 6:
play, buy, worth, hour, sale, money, free, best, dlc, want
buy, worth, play, ho

---

Get most representative docs

In [72]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics(num_topics=-1)
# ...then create a empty list per topic to collect the docs:
docs_per_topic = {topic_id: [] for (topic_id, _) in all_topics}

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

In [73]:
for doc_list in docs_per_topic.values():
    doc_list.sort(key=lambda id_and_score: id_and_score[1], reverse=True)

In [74]:
top_N_docs = 10

for i in range(len(docs_per_topic)):
    print(docs_per_topic[i][:top_N_docs])

[(28144, 0.9099976), (6243, 0.8874805), (10905, 0.87141025), (73356, 0.871363), (18242, 0.8507218), (74078, 0.8499957), (48198, 0.8499941), (72287, 0.84998393), (50937, 0.849981), (63878, 0.84997684)]
[(32897, 0.99470586), (79127, 0.98988754), (14316, 0.98676443), (73523, 0.9823496), (16644, 0.9709629), (12890, 0.9470606), (18204, 0.9437416), (60389, 0.9399906), (57561, 0.93570083), (14517, 0.9356924)]
[(74128, 0.9917308), (8505, 0.9181381), (15936, 0.91812587), (27581, 0.9099801), (55120, 0.909955), (93267, 0.9092642), (73277, 0.8999884), (64982, 0.89994556), (91174, 0.899886), (26409, 0.8874884)]
[(90095, 0.9898874), (82831, 0.9181864), (56893, 0.9099874), (54441, 0.8999977), (94498, 0.8999905), (11165, 0.89998996), (86456, 0.89996535), (70824, 0.8875036), (70346, 0.88750225), (52783, 0.8874985)]
[(79123, 0.924986), (70970, 0.9181628), (19189, 0.9181544), (88769, 0.91812855), (41025, 0.90998244), (26866, 0.90997607), (11562, 0.9099693), (56077, 0.90994865), (84377, 0.8999949), (54420

In [75]:
reviews_df

,recommendationid,review,timestamp_created,voted_up,steam_purchase,received_for_free,review_lemmatized
0,157967184,loved every bethesda say particularly even lik...,2024-02-10 03:01:14,-1,True,False,"[love, every, bethesda, say, particularly, eve..."
1,157967139,game feels enjoy replaying story lines charact...,2024-02-10 03:00:23,-1,True,False,"[game, feel, enjoy, replay, story, line, chara..."
2,157966777,initially disappointed game come along way ser...,2024-02-10 02:52:15,1,True,False,"[initially, disappointed, game, come, along, w..."
3,157965911,kewl groundbreaking space game game space ship...,2024-02-10 02:33:16,1,True,False,"[kewl, groundbreaking, space, game, game, spac..."
4,157964949,took skyrim space little seriously ran engine ...,2024-02-10 02:15:34,1,True,False,"[take, skyrim, space, little, seriously, ran, ..."
...,...,...,...,...,...,...,...
96729,145736027,hours played still loving much fun far finishe...,2023-09-06 00:00:32,1,True,False,"[hour, play, still, love, much, fun, far, fini..."
96730,145736019,skyrim,2023-09-06 00:00:31,1,True,False,[skyrim]
96731,145736018,fallout space space sim shit,2023-09-06 00:00:31,1,False,False,"[fallout, space, space, sim, shit]"
96732,145736016,starfield bethesda rpg essence mean immense fr...,2023-09-06 00:00:30,1,False,False,"[starfield, bethesda, rpg, essence, mean, imme..."


In [76]:
# use the ID to retrieve the top docs, and copy them to a file for inspection

# retrieve the original text
df_original_texts = []
for topic_id in docs_per_topic.keys():
    t = reviews_df.iloc[[doc_id for doc_id, _ in docs_per_topic[topic_id][:top_N_docs]]]
    t['topic_id'] = topic_id        # store the topic id

    df_original_texts.append(t)

df_original_texts = pd.concat(df_original_texts)
df_original_texts

/tmp/ipykernel_28452/399986568.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['topic_id'] = topic_id        # store the topic id
/tmp/ipykernel_28452/399986568.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['topic_id'] = topic_id        # store the topic id
/tmp/ipykernel_28452/399986568.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,recommendationid,review,timestamp_created,voted_up,steam_purchase,received_for_free,review_lemmatized,topic_id
28532,148404424,walk walk walk walk walk walk walk walk walk,2023-10-17 20:13:14,-1,True,False,"[walk, walk, walk, walk, walk, walk, walk, wal...",0
6312,154825808,forced story twice see got better high levels ...,2023-12-30 18:07:08,-1,True,False,"[force, story, twice, see, got, well, high, le...",0
11041,153559988,starfield like watching magician pull rabbit h...,2023-12-14 08:12:50,-1,True,False,"[starfield, like, watch, magician, pull, rabbi...",0
74477,145874649,tried grab dart board stole instead wife left ...,2023-09-07 20:41:53,1,True,False,"[tried, grab, dart, board, stole, instead, wif...",0
18479,151258244,like stop look around wondrous sights unexplor...,2023-11-23 07:12:58,1,True,False,"[like, stop, look, around, wondrous, sight, un...",0
...,...,...,...,...,...,...,...,...
29282,148295850,good depth experience lot learn much experienc...,2023-10-15 18:29:20,1,True,False,"[good, depth, experience, lot, learn, much, ex...",9
47870,146563734,needs lot mod support community good game,2023-09-19 00:36:53,-1,True,False,"[need, lot, mod, support, community, good, game]",9
73837,145881085,needed good game get lost thanks todd team,2023-09-07 23:10:06,1,True,False,"[need, good, game, get, lose, thanks, todd, team]",9
27588,148549937,rate starfield ten plenty features story good ...,2023-10-20 16:28:58,1,True,False,"[rate, starfield, ten, plenty, feature, story,...",9


In [77]:
# print out the original texts as a log

for topic_id in docs_per_topic.keys():
    print(f'Topic {topic_id}:')
    print()
    t = reviews_df.iloc[[doc_id for doc_id, _ in docs_per_topic[topic_id][:top_N_docs]]]
    for index, row in t.iterrows():
        print(f'Doc {index}:')
        print(row['review'])
        print()
    print()

Topic 0:

Doc 28532:
walk walk walk walk walk walk walk walk walk

Doc 6312:
forced story twice see got better high levels story hope like loadscreens every damn seconds

Doc 11041:
starfield like watching magician pull rabbit hat close eyes

Doc 74477:
tried grab dart board stole instead wife left bgs game

Doc 18479:
like stop look around wondrous sights unexplored chug beer everytime need stop recharge carry must carry around junk looking sarah shut

Doc 75208:
hey finally awake trying jump systems right flew right uc ambush us smuggler

Doc 48898:
see mountain gonna walk enemies chasing encumbered stuff need

Doc 73391:
game sucks like sitting top lb woman stomach full smegma cheetos leftovers

Doc 51695:
game made cry like little girl pound man

Doc 64845:
press random button enter simulator


Topic 1:

Doc 33338:
crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes c

In [78]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics(num_topics=-1)
# ...then create a empty list per topic to collect the docs:
docs_per_topic = {topic_id: [] for (topic_id, _) in all_topics}

docs_top1_per_topic = {topic_id: [] for (topic_id, _) in all_topics}

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
        
    topic_id_max = -1; max_score = float('-inf')

    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

        if score > max_score:
            max_score = score
            topic_id_max = topic_id
    
    docs_top1_per_topic[topic_id_max].append((doc_id, max_score))

In [79]:
df_eval_topic_freq = pd.DataFrame(
    {
        'topic_id': [topic_id for topic_id in docs_top1_per_topic.keys()],
        'topic_freq': [len(docs) for docs in docs_top1_per_topic.values()]
    }
)

df_eval_topic_freq

,topic_id,topic_freq
0,0,3786
1,1,12005
2,2,18923
3,3,12170
4,4,9647
5,5,6356
6,6,5094
7,7,2859
8,8,22720
9,9,1681


In [80]:
top_n = 10
df_original_texts.to_pickle(eval_results_external_folder_path.joinpath(f'df_eval_top_{top_n}.pkl'))

In [81]:
# also need to save the top N keywords for each topic as json
with open(eval_results_external_folder_path.joinpath(f'top_{top_N_words}_keywords.json'), 'w') as f:
    json.dump(topic_keywords, f, indent=2)

In [82]:
# save the topic frequency  (top 1 prob)
df_eval_topic_freq.to_pickle(
    eval_results_external_folder_path.joinpath(f'df_eval_topic_freq.pkl')
)

---

LLM topic naming

Script copied from lda_eval_quali.ipynb

In [83]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"          # disable huggingface warning

# device check
import platform
import torch
if platform.system() == 'Linux' or platform.system() == 'Windows':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('mps')        # m-series machine

print(device)

cuda


In [84]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

In [85]:
llm = Ollama(model="llama2")        # assuming the port is 11434

In [86]:
# prompt engineering
system_message = "You are a player of the game who is reading the reviews about the game."

human_template = \
'''Create a name for a topic given the topic's keywords and some most representative reviews of the topic. Output a label for the topic in less than 5 words. Output "NA" if the topic is not clear. Do not output other text. 

The top keywords of the topic is: \'\'\'{topic_keywords}\'\'\'. 

The most representative reviews of the topic are: \'\'\'{topic_reviews}\'\'\'.'''

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_template)
])

chain = chat_prompt | llm

In [87]:
new_topic_labels = {}
randomed_topic_reviews = {}

In [88]:
import random
import time

N_times = 5

topic_ids = list(docs_per_topic.keys())           # also generate the labels for the outlier topic, as its part of the topic_labels_ attribute

# new_topic_labels = {}
# randomed_topic_reviews = {}

for topic_id in topic_ids:
    _topic_keywords = topic_keywords[topic_id]

    temp_disable_char_limit = False
    _count = 0

    # time.sleep(1)

    _reviews_df = df_original_texts[df_original_texts['topic_id'] == topic_id]
    for i in range(N_times):
        if new_topic_labels.get(topic_id, {}).get(f"call_{i}", None) is not None:
            print(f'{topic_id:02}_call{i}: {new_topic_labels[topic_id][f"call_{i}"]}')
            continue

        while True:
            if _count > 20:
                temp_disable_char_limit = True

            _sampled_reviews_df = _reviews_df.sample(n=2, replace=False)

            # check the length of the topic reviews so that the llm won't be overloaded
            # 5000 character limits
            check_bool = _sampled_reviews_df.apply(lambda x: len(x['review']) < 5000, axis=1)
            
            if temp_disable_char_limit:
                break
            
            if all(check_bool):
                break
            else:
                _count += 1
        
        topic_reviews = _sampled_reviews_df['review'].values
        print(topic_reviews)

        result = chain.invoke(
            {
                "topic_keywords": _topic_keywords,
                "topic_reviews": topic_reviews
            }
        )

        print(f'{topic_id:02}_call{i}: {result}')
            
        if topic_id not in new_topic_labels:
            new_topic_labels[topic_id] = {}
            randomed_topic_reviews[topic_id] = {}

        new_topic_labels[topic_id][f"call_{i}"] = result
        randomed_topic_reviews[topic_id][f"call_{i}"] = {
            'reviews': topic_reviews.tolist(),
            "recommendationid": _sampled_reviews_df['recommendationid'].values.tolist()
        }


    print('\n')

['game sucks like sitting top lb woman stomach full smegma cheetos leftovers'
 'see mountain gonna walk enemies chasing encumbered stuff need']
00_call0: 
Topic Label: Kill Game
['hey finally awake trying jump systems right flew right uc ambush us smuggler'
 'see mountain gonna walk enemies chasing encumbered stuff need']
00_call1: 
Label: Action-Packed Shooter
['see mountain gonna walk enemies chasing encumbered stuff need'
 'press random button enter simulator']


00_call2: 
Topic: Action-Packed Shooter
Label: "Kill or Die"
['game made cry like little girl pound man'
 'starfield like watching magician pull rabbit hat close eyes']
00_call3: Topic Label: Kill
['press random button enter simulator'
 'hey finally awake trying jump systems right flew right uc ambush us smuggler']
00_call4: Topic Label: Kill


['crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes crashes cra

In [89]:
# save the topic labels

llm_generation_result = {
    'new_topic_labels': new_topic_labels,
    'randomed_topic_reviews': randomed_topic_reviews
}

with open(eval_results_external_folder_path.joinpath('llm_generation_result.json'), 'w') as f:
    json.dump(llm_generation_result, f, indent=2)